# Computer Vision
This notebook includes some examples of techniques used to analyze images - which is a common requirement in AI solutions.

## Manipulating Images
As far as computers are concerned, images are simply numerical data representations. You can use statistical techniques to manipulate and analyze the numerical properties of images.

### Load an Image
Let's start by loading a JPG file and examining its properties. Run the following cell to load and display an image.

In [ ]:
%matplotlib inline
from matplotlib.pyplot import imshow
from PIL import Image
import numpy as np
import skimage.color as sc

!curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/graeme2.jpg -o img.jpg

i = np.array(Image.open('img.jpg'))
imshow(i)

### Examine Numerical Properties of the Image
You can clearly see that this is an image, but how does the computer interpret the data?

Run the cell below to determine the data type of the image.

In [ ]:
type(i)

The image data is actually stored as a multi-dimensional array.

Let's see what data type the array elements are:

In [ ]:
i.dtype

OK, so the array consists of 8-bit integer values. In other words, whole numbers between 0 and 255.

Now let's examine the shape of the array:

In [ ]:
i.shape

So the image data is a three dimensional 433 x 650 x 3 array.

This is a RGB color JPG image sized 433 x 650 pixels. The image includes pixel values for red, green, and blue color channels. 

To keep things simple, let's convert the image to a greyscale image so we only have one color channel dimension to deal with:

In [ ]:
i_mono = sc.rgb2gray(i)
imshow(i_mono, cmap='gray')
i_mono.shape

### View Pixel Value Distributions
Let's look at the distribution of pixel values in the image. Ideally, the image should have relatively even distribution of values, indicating good contrast and making it easier to extract analytical information.

An easy way to check this is to plot a histogram.

In [ ]:
def im_hist(img):
    import matplotlib.pyplot as plt    
    fig = plt.figure(figsize=(8, 6))
    fig.clf()
    ax = fig.gca()    
    ax.hist(img.flatten(), bins = 256)
    plt.show()

im_hist(i_mono)

Another useful way to visualize the statistics of an image is as a cumulative distribution function (CDF) plot. Ideally, this should result in a fairly straight diagonal line.

In [ ]:
def im_cdf(img):
    import matplotlib.pyplot as plt    
    fig = plt.figure(figsize=(8, 6))
    fig.clf()
    ax = fig.gca()    
    ax.hist(img.flatten(), bins = 256, cumulative=True)
    plt.show()
    
im_cdf(i_mono)

The histogram and CDF for our image show pretty uneven distribution. Ideally we should equalize the values in the image to improve its analytical value.

### Equalize the Image
Histogram equalization is often used to improve the statistics of images. In simple terms, the histogram equalization algorithm attempts to adjust the pixel values in the image to create a more uniform distribution. The code in the cell below uses the  **exposure.equalize_hist** method from the **skimage** package to equalize the image.  

In [ ]:
from skimage import exposure

i_eq = exposure.equalize_hist(i_mono)
imshow(i_eq, cmap='gray')

Now let's see what that's done to the histogram and CDF plots:

In [ ]:
im_hist(i_eq)
im_cdf(i_eq)

The pixel intensities are more evenly distributed in the equalized image.

## Denoising with Filters

Often images need to be cleaned up to remove "salt and pepper" noise.

### Add Some Random Noise
Let's add some random noise to our image - such as you might see in a photograph taken in low light or at a low resolution.

In [ ]:
import skimage
i_n = skimage.util.random_noise(i_eq)
imshow(i_n, cmap="gray")

### Using a Gaussian Filter
A Gaussian filter applies a weighted average (mean) value for pixels based on the pixels that surround them.

In [ ]:
def gauss_filter(im, sigma = 2):
    from scipy.ndimage.filters import gaussian_filter as gf
    import numpy as np
    return gf(im, sigma = sigma)   
i_g = gauss_filter(i_n)
imshow(i_g, cmap="gray")

### Using a Median Filter
The Gaussian filter results in a blurred image - we could try a median filter, which as the name suggests applies the median value to pixels based on the pixels around them.

In [ ]:
def med_filter(im, size = 2):
    from scipy.ndimage.filters import median_filter as mf
    import numpy as np
    return mf(im, size = size)     
i_m = med_filter(i_n)
imshow(i_m, cmap="gray")

### Extract Features
Now that we've done some initial processing of the image to improve its statistics for analysis, we can start to extract features from it.
#### Sobel Edge Detection
As a first step in extracting features, you will apply the Sobel edge detection algorithm. This finds regions of the image with large gradient values in multiple directions. Regions with high omnidirectional gradient are likely to be edges or transitions in the pixel values. 

The code in the cell below applies the Sobel algorithm to the median filtered image, using these steps:

    1. Convert the color, rgb, image to grayscale (in this case, the image is already grayscale - but you should always do this because using a grayscale image simplifies the gradient calculation since it is two dimensional.
    2. Computer the gradient in the x and y (horizontal and vertical) directions. 
    3. Compute the magnitude of the gradient.
    4. Normalize the gradient values. 


In [ ]:
def edge_sobel(image):
    from scipy import ndimage
    import skimage.color as sc
    import numpy as np
    image = sc.rgb2gray(image) # Convert color image to gray scale
    dx = ndimage.sobel(image, 1)  # horizontal derivative
    dy = ndimage.sobel(image, 0)  # vertical derivative
    mag = np.hypot(dx, dy)  # magnitude
    mag *= 255.0 / np.amax(mag)  # normalize (Q&D)
    mag = mag.astype(np.uint8)
    return mag

i_edge = edge_sobel(i_m)
imshow(i_edge, cmap="gray")

Now let's try with the more blurred gaussian filtered image.

In [ ]:
i_edge = edge_sobel(i_g)
imshow(i_edge, cmap="gray")

Note that the lines are more pronounced. Although a gaussian filter makes the image blurred to human eyes, this blurring can actually help accentuate contrasting features.

#### Harris Corner Detection
Another example of a feature extraction algorithm is corner detection. In simple terms, the Harris corner detection algorithm locates regions of the image with large changes in pixel values in all directions. These regions are said to be corners. The Harris corner detector is paired with the **corner_peaks** method. This operator filters the output of the Harris algorithm, over a patch of the image defined by the span of the filters, for the most likely corners.

In [ ]:
# Function to apply the Harris corner-detection algorithm to an image
def corner_harr(im, min_distance = 20):
    from skimage.feature import corner_harris, corner_peaks
    mag = corner_harris(im)
    return corner_peaks(mag, min_distance = min_distance)

# Find the corners in the median filtered image with a minimum distance of 20 pixels
harris = corner_harr(i_m, 20)

print (harris)

# Function to plot the image with the harris corners marked on it
def plot_harris(im, harris, markersize = 20, color = 'red'):
    import matplotlib.pyplot as plt
    import numpy as np
    fig = plt.figure(figsize=(6, 6))
    fig.clf()
    ax = fig.gca()    
    ax.imshow(np.array(im).astype(float), cmap="gray")
    ax.plot(harris[:, 1], harris[:, 0], 'r+', color = color, markersize=markersize)
    return 'Done'  

plot_harris(i_m, harris)

The corner detection algorithm has identified the eyes in the image.

## Using the Computer Vision API
The techniques used so far in this notebook show how you can perform simple image manipulation and apply some popular algorithms to analyze images. More complex image analysis capabilities are encapsulated in the Computer Vision API cognitive service.

### Create a Computer Vision API Service
To provision a Computer Vision API service in your Azure subscription, Follow these steps:

1. Open another browser tab and navigate to https://portal.azure.com.
2. Sign in using your Microsoft account.
3. Click **+ New**, and in the **AI + Cognitive Services** category, click **See all**.
4. In the list of cognitive services, click **Computer Vision API**.
5. In the **Computer Vision API** blade, click **Create**.
6. In the **Create** blade, enter the following details, and then click **Create**
  * **Name**: A unique name for your service.
  * **Subscription**: Your Azure subscription.
  * **Location**: Choose the Azure datacenter location where you want to host your service.
  * **Pricing tier**: Choose the F0 pricing tier.
  * **Resource Group**: Choose the existing resource group you created in the previous lab (or create a new one if you didn't complete the previous lab)
  * Read the notice about the use of your data, and select the checkbox.
7. Wait for the service to be created.
8. When deployment is complete, click **All Resources** and then click your Computer Vision service to open its blade.
9. In the blade for your Computer Vision service, note the **Endpoint** URL. Then assign the base URI (*location*.api.cognitive.microsoft.com) for your service to the **visionURI** variable in the cell below.
10. In the blade for your Computer Vision service, click **Keys** and then copy **Key 1** to the clipboard and paste it into the **visionKey** variable assignment value in the cell below. 
11. Run the cell below to assign the variables.


In [ ]:
visionURI = 'westeurope.api.cognitive.microsoft.com'
visionKey = '369ec0f9e52a4695a3b0c133000e81c2'#'1234567abcdefg'

### Get An Image from a URL
Let's start with the same image we analyzed previously.

Run the code in the cell below to retrieve the original color image from its URL:

In [ ]:
%matplotlib inline
from matplotlib.pyplot import imshow
from PIL import Image
import requests
from io import BytesIO

img_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/graeme2.jpg'

# Get the image and show it
response = requests.get(img_url)
img = Image.open(BytesIO(response.content))
imshow(img)

### Use the Computer Vision API to Get Image Features
The Computer Vision API uses a machine learning model that has been trtained with millions of images. It can extract features from images and return a suggested description, as well as details about the image file and a suggested list of "tags" that apply to it.

Run the cell below to see what caption the Computer Vision API suggests for the image above.

In [ ]:
def get_image_features(img_url):
    import http.client, urllib.request, urllib.parse, urllib.error, base64, json

    headers = {
        # Request headers.
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': visionKey,
    }

    params = urllib.parse.urlencode({
        # Request parameters. All of them are optional.
        'visualFeatures': 'Categories,Description,Color',
        'language': 'en',
    })

    body = "{'url':'" + img_url + "'}"

    try:
        # Execute the REST API call and get the response.
        conn = http.client.HTTPSConnection(visionURI)
        conn.request("POST", "/vision/v1.0/analyze?%s" % params, body, headers)
        response = conn.getresponse()
        data = response.read().decode("UTF-8")

        # 'data' contains the JSON response.
        parsed = json.loads(data)
        if response is not None:
            return parsed
        conn.close()


    except Exception as e:
        print('Error:')
        print(e)
        
jsonData = get_image_features(img_url)
desc = jsonData['description']['captions'][0]['text']
print(desc)

The description is reasonably, if not exactly, appropriate.

Run the cell below to see the full JSON response, including image properties and suggested tags.

In [ ]:
import json

# View the full details returned
print (json.dumps(jsonData, sort_keys=True, indent=2))

Let's try with a different image:

In [ ]:
img_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/uke.jpg'

# Get the image and show it
response = requests.get(img_url)
img = Image.open(BytesIO(response.content))
imshow(img)
jsonData = get_image_features(img_url)
desc = jsonData['description']['captions'][0]['text']
print(desc)

How about something a little more complex?

In [ ]:
img_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/soccer.jpg'

# Get the image and show it
response = requests.get(img_url)
img = Image.open(BytesIO(response.content))
imshow(img)
jsonData = get_image_features(img_url)
desc = jsonData['description']['captions'][0]['text']
print(desc)

## Using the Face API
While the Computer Vision API is useful for general image analysis, the Face API offers specific functions for analyzing faces in images. This can be useful in a variety of AI scenarios.

### Create a Face API Service
To provision a Computer Vision API service in your Azure subscription, Follow these steps:

1. Open another browser tab and navigate to https://portal.azure.com.
2. Sign in using your Microsoft account.
3. Click **+ New**, and in the **AI + Cognitive Services** category, click **See all**.
4. In the list of cognitive services, click **Face API**.
5. In the **Face API** blade, click **Create**.
6. In the **Create** blade, enter the following details, and then click **Create**
  * **Name**: A unique name for your service.
  * **Subscription**: Your Azure subscription.
  * **Location**: Choose the Azure datacenter location where you want to host your service.
  * **Pricing tier**: Choose the F0 pricing tier.
  * **Resource Group**: Choose the existing resource group you created in the previous lab (or create a new one if you didn't complete the previous lab)
  * Read the notice about the use of your data, and select the checkbox.
7. Wait for the service to be created.
8. When deployment is complete, click **All Resources** and then click your Face service to open its blade.
9. In the blade for your Face service, copy the *full* **Endpoint** URL (including the *https* prefix and */face/v1.0/* path), and paste it into the **faceURI** variable assignment value in the cell below.
10. In the blade for your Face service, click **Keys** and then copy **Key 1** to the clipboard and paste it into the **faceKey** variable assignment value in the cell below. 
11. Run the cell below to assign the variables.

In [ ]:
faceURI = "https://westeurope.api.cognitive.microsoft.com/face/v1.0/"
faceKey = "12345678abcdegfh"

The Face API has a Python SDK, which you can install as a package. This makes it easier to work with.

Run the following cell to install the Face API SDK and the pillow package (which is used to work with images)

In [ ]:
!pip install cognitive_face
!pip install pillow

Now you're ready to use the Face API. First, let's see if we can detect a face in an image:

In [ ]:
import cognitive_face as CF

# Set URI and Key
CF.BaseUrl.set(faceURI)
CF.Key.set(faceKey)


# Detect faces in an image
img_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/graeme1.jpg'
result = CF.face.detect(img_url)
print (result)

The Face API has detected one face, and assigned it an ID. It also returns the coordinates for the top left corner and the width and height for the rectangle within which the face is detected.

Run the cell below to show the rectange on the image.

In [ ]:
%matplotlib inline
import requests
from io import BytesIO
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw

# Get the image
response = requests.get(img_url)
img = Image.open(BytesIO(response.content))

# Add rectangles for each face found
color="blue"
if result is not None:
    draw = ImageDraw.Draw(img) 
    for currFace in result:
        faceRectangle = currFace['faceRectangle']
        left = faceRectangle['left']
        top = faceRectangle['top']
        width = faceRectangle['width']
        height = faceRectangle['height']
        draw.line([(left,top),(left+width,top)],fill=color, width=5)
        draw.line([(left+width,top),(left+width,top+height)],fill=color , width=5)
        draw.line([(left+width,top+height),(left, top+height)],fill=color , width=5)
        draw.line([(left,top+height),(left, top)],fill=color , width=5)

# show the image
imshow(img)


As well as detecting the face, the Face API assigned an ID to this face. The ID is retained by the service for a while, enabling you to reference it. Run the following cell to see the ID assigned to the face that has been detected:

In [ ]:
face1 = result[0]['faceId']
print ("Face 1:" + face1)

One useful thing you can do with the face ID is, is to use it to compare another image and see if a matching face is found. This kind of facial comparison is common in a variety of security / user authentication scenarios.

Let's try it with another image of the same person:

In [ ]:
# Get the image to compare
img2_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/graeme2.jpg'
response2 = requests.get(img2_url)
img2 = Image.open(BytesIO(response2.content))

# Detect faces in a comparison image
result2 = CF.face.detect(img2_url)

# Assume the first face is the one we want to compare
if result2 is not None:
    face2 = result2[0]['faceId']
    print ("Face 2:" + face2)

def verify_face(face1, face2):
    # By default, assume the match is unverified
    verified = "Not Verified"
    color="red"

    # compare the comparison face to the original one we retrieved previously
    verify = CF.face.verify(face1, face2)

    # if there's a match, set verified and change color to green
    if verify['isIdentical'] == True:
        verified = "Verified"
        color="lightgreen"

    # Display the second face with a red rectange if unverified, or green if verified
    draw = ImageDraw.Draw(img2) 
    for currFace in result2:
        faceRectangle = currFace['faceRectangle']
        left = faceRectangle['left']
        top = faceRectangle['top']
        width = faceRectangle['width']
        height = faceRectangle['height']
        draw.line([(left,top),(left+width,top)] , fill=color, width=5)
        draw.line([(left+width,top),(left+width,top+height)] , fill=color, width=5)
        draw.line([(left+width,top+height),(left, top+height)] , fill=color, width=5)
        draw.line([(left,top+height),(left, top)] , fill=color, width=5)

    # show the image
    imshow(img2)

    # Display verification status and confidence level
    print(verified)
    print ("Confidence Level: " + str(verify['confidence']))

verify_face(face1, face2)

The service has matched the face in a similar photo, with a reasonably high confidence level.

But what about the same face in a different photo - maybe with a stylish goatee beard and sunglasses?:

In [ ]:
# Get the image to compare
img2_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/graeme3.jpg'
response2 = requests.get(img2_url)
img2 = Image.open(BytesIO(response2.content))

# Detect faces in a comparison image
result2 = CF.face.detect(img2_url)

# Assume the first face is the one we want to compare
face2 = result2[0]['faceId']
print ("Face 2:" + face2)

verify_face(face1, face2)


Again, the face is matched - but with lower confidence reflecting the differences in the image.

What if we try to match the original face to a different person?

In [ ]:
# Get the image to compare
img2_url = 'https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/satya.jpg'
response2 = requests.get(img2_url)
img2 = Image.open(BytesIO(response2.content))

# Detect faces in a comparison image
result2 = CF.face.detect(img2_url)

# Assume the first face is the one we want to compare
face2 = result2[0]['faceId']
print ("Face 2:" + face2)

verify_face(face1, face2)


No match!

## Using the Video Indexer API
So far we've used various Cognitive Services APIs to work with static images. However, you can also use the video indexer API to work with video.

### Create a Video Indexer API Subscription
At the time of writing, the **Video Indexer** API is in preview, and not available in the Azure portal. To access the preview service, follow these steps:

1. Open another browser tab and navigate to https://videobreakdown.portal.azure-api.net/.
2. Sign in using the Microsoft account associated with your Azure subscription. If prompted, grant the required permissions to the service.
3. Enter the required details to sign up.
4. On the **Products** page, click **Free Preview** ansd then click **Subscribe**. Then agree to the terms of use and click **Confirm**.
5. Next to the **Primary key** that has been generated, click **Show**. Then copy and paste the key to set the **apiKey** variable in the cell below.
6. Run the cell below to assign the variable.


In [ ]:
apiKey = "12345abcdefg"

### Upload a Video for Processing
In this exercise, you will use the **Video Indexer** API to analyze a short video. First, you must upload the video to the Video Indexer so it can be processed. To do this, you can call the **Upload** operation, which uploads the video and starts an asynchronous indexing process.

Run the cell below to do this.

In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

# We'll upload this video from GitHub to the Video Indexer
video_url='https://github.com/MicrosoftLearning/AI-Introduction/raw/master/files/vid.mp4'

headers = {
    # Request headers
    'Content-Type': 'multipart/form-data',
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
    # Request parameters
    'name': 'vid',
    'privacy': 'Private',
    'videoUrl': video_url,
    'language': 'en-US',
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("POST", "/Breakdowns/Api/Partner/Breakdowns?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    # The response is a byte array - convert to a string and remove quotes
    vidId = data.decode("UTF-8").replace('"', '')
    print("Breakdown ID: " + vidId)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

### Check Processing Status
The **Upload** operation returned an ID for the video breakdown. You can use this to track the status of the asynchronous indexing process.

Run the following cell to get the status, and keep running it until the status indicates that the video has been processed.

In [ ]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("GET", "/Breakdowns/Api/Partner/Breakdowns/%s/State?%s" % (vidId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))


### View the Video
The Video Indexer provides a *Player* widget that you can use to embed the video into a web application.

Run the cell below to watch the video.

In [ ]:
from IPython.core.display import HTML

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("GET", "/Breakdowns/Api/Partner/Breakdowns/%s/PlayerWidgetUrl?%s" % (vidId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    vidUrl = data.decode("UTF-8").replace('"', '')
    print (vidUrl)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
HTML('<iframe width=900 height=600 src="%s"/>' % vidUrl )

### View the Video Breakdown
After the video has been indexed, you can retrieve the detailed breakdown of insights that were found by calling the **Get Breakdown** operation.

Run the following cell to get the video breakdown in JSON format.

In [ ]:
import json

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
    # Request parameters
    'language': 'en-US',
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("GET", "/Breakdowns/Api/Partner/Breakdowns/%s?%s" % (vidId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    strData = data.decode("UTF-8")
    jData = json.loads(strData)
    print (json.dumps(jData, sort_keys=True, indent=2))
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))



### Get Details of Faces Identified in the Video
The breakdown includes details of faces that were identified in the video.

Run the following cell to view these details.

In [ ]:
print(json.dumps(jData["summarizedInsights"]["faces"], sort_keys=True, indent=2))

### View a Face Thumbnail
In this case, a single unidentified face was detected at around 8.4 seconds into the video.

Run the following cell to see a thumbnail of the face.

In [ ]:
faceUrl = jData["summarizedInsights"]["faces"][0]["thumbnailFullUrl"]
print(faceUrl)
HTML('<img src="%s"/>' % faceUrl )

### View (and Edit) People Insights
That face looks familiar!
You can use the *Insights* widget to allow users to view and edit insights, inclyding those relating to people in the vide.

Run the following cell to load the insights widget for people, and then edit the insights to change the name of the person identified to "Graeme".

In [ ]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
    # Request parameters
    'widgetType': 'People',
    'allowEdit': 'True',
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("GET", "/Breakdowns/Api/Partner/Breakdowns/%s/InsightsWidgetUrl?%s" % (vidId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    iUrl = data.decode("UTF-8").replace('"', '')
    print(iUrl)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

HTML('<iframe width=900 height=600 src="%s"/>' % iUrl )

### Reload Breakdown and Check Updated Face Details
To verify that your updates have been saved, run the following cell to reload the breakdown and display the face metadata.

In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
    # Request parameters
    'language': 'en-US',
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("GET", "/Breakdowns/Api/Partner/Breakdowns/%s?%s" % (vidId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    strData = data.decode("UTF-8")
    jData = json.loads(strData)
    print(json.dumps(jData["summarizedInsights"]["faces"], sort_keys=True, indent=2))
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))


The name you specified has been assigned to this person, and if this face is detected in any future videos, it will be identified as the same person in this video. 